In [ ]:
train = pd.read_csv("train_states.csv",low_memory=False) # Import the data

In [ ]:
print(train.shape)
train = train.drop_duplicates()
print(train.shape) #no duplicates

# The problem is that not all stores have full data available

#this shows how many stores have data available to all the dates
np.sum(train.groupby('Store')['DayOfWeek'].count() != 942) 
#for simplification, let us delete all the stores with incomplete data
#first find out, which store these are
indices = np.where((train.groupby('Store')['DayOfWeek'].count() != 942) == True)
indices = list(indices) #tuple needs to be converted to list
#caution, previous fct. counts from 0, so to get store number we need to add 1
indices = [x+1 for x in indices]
#we need to simplify list containing one array into array
indices = [item for sublist in indices for item in sublist]

#And we can finally filter only the complete stores
train = train[~train['Store'].isin(indices)] #the wave is a negation sign

In [ ]:
train = train.sort_values(by=['Store','Date']) #sort by store and date
len(train[train['Store'] == 1]) #942 dates

In [ ]:
#Delete unimportant variables
train = train.drop\
        (['is_month_start','is_month_end','is_quarter_start','is_quarter_end','State', #same for all the images
          'week_of_year', #partially identifiable by other variables
          'day_of_year', #identifiable by other variables       
         'day'], axis = 1) #day is partially already in the time dimension

In [ ]:
def dummify(data,data_and_column,prefix):
    dummies = pd.get_dummies(data_and_column, prefix=prefix).iloc[:,1:] #creates dummy variables (12-1 dummy variables with Month_1 as default)
    dataframe = pd.concat([data,dummies], axis = 1)
    return dataframe
#dummies
#train
train = dummify(train,train.DayOfWeek,'DayOfWeek')
train = train.drop('DayOfWeek', axis = 1)
train = dummify(train,train.StateHoliday,'StateHoliday')
train = train.drop('StateHoliday', axis = 1)
train = dummify(train,train.quarter,'Q')  #quarter will also be delete, but later
train = train.drop('quarter', axis = 1)
#we will first change month into months of the quarter 
#(as we will be using quarters later)
#NOT A NICE CHUNK OF CODE
train.month = [1 if e == 4  else e for e in train.month]
train.month = [1 if e == 7  else e for e in train.month]
train.month = [1 if e == 10 else e for e in train.month]
train.month = [2 if e == 5  else e for e in train.month]
train.month = [2 if e == 8  else e for e in train.month]
train.month = [2 if e == 11 else e for e in train.month]
train.month = [3 if e == 6  else e for e in train.month]
train.month = [3 if e == 9  else e for e in train.month]
train.month = [3 if e == 12 else e for e in train.month]
train = dummify(train,train.month,'month')
train = train.drop('month', axis = 1)
#train = train.drop('Date', axis = 1) we need Date for reshaping
train['Date'] = train['Date'].str.replace("-","").astype(int) #convert date to numeric

In [ ]:
column_names = pd.DataFrame(list(train))
column_names = column_names.transpose()
np.savetxt("column_names.csv",column_names,fmt="%s",delimiter=",")
#ok now i have a ordered table with the whole time series with each store having 942 rows (days) and 23 variables
#after complete info about first store (942 rows), the second store follows, and so on
#now I need 90 days time series, with 91th day as the label and then move it by 1 day